# Lab #3
In this laboratory, you will learn more about frequent itemsets and association rules.

## Online Retail Data set
The Online Retail Data Set [1] is a dataset made available on the UCI Machine Learning repository. It which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based online retail.

https://github.com/dbdmg/data-science-lab/raw/master/datasets/online_retail.csv

Each of the 541,909 rows contains an item that has been purchased by someone. Items can be grouped into invoices (you can think of these as receipts), where each invoice has been issued for a specific buyer, and can contain multiple items.
The columns contained in the CSV file are the following:
- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'C’, it indicates a cancellation.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.

### Association rules from frequent itemses

**1 - Loading and cleaning the dataset**

Loading the dataset and removing invalid rows and those having an InvoiceNo that starts with "C", as they indicate that the invoice is about a cancelled purchase.

In [1]:
import pandas as pd
import re
import csv

In [2]:
with open ("online_retail.csv") as f:
    reader = csv.reader(f)
    HretailDs=list(reader)
    
headers=HretailDs[0]
wholeRetailDs=HretailDs[1:]

In [3]:
#Removing all the rows that has the invoice attribute that start with "C"
#Removingo other errors in the dataset
retailDs = []
for row in wholeRetailDs:
    if not row[0].startswith('C') and not re.search("(?=.*[a-z])|^\\s*$|^\\?", row[2].strip()):#tolgo tutti quelli che iniziano per C o che hanno delle minuscole
        retailDs.append(row)
len(retailDs)

528464

**2 - Aggregate at "invoice" level.**

Now that you have a dataset of items, you should aggregate it at an “invoice” level. For each invoice (identified by InvoiceNo) there can be multiple items (from multiple rows) in the dataset. For each invoice, you should build a list of all items belonging to it.

In [4]:
#Creating a dict to associate all the rows with the same InvoiceNo
tmpDict = {}
for row in retailDs:
    if (row[0] in tmpDict.keys()):
        tmpDict[row[0]].append(row[2].strip())
    else:
        tmpDict[row[0]]=[row[2].strip()]
invoices = list(tmpDict.values())
invoices[0]

['WHITE HANGING HEART T-LIGHT HOLDER',
 'WHITE METAL LANTERN',
 'CREAM CUPID HEARTS COAT HANGER',
 'KNITTED UNION FLAG HOT WATER BOTTLE',
 'RED WOOLLY HOTTIE WHITE HEART.',
 'SET 7 BABUSHKA NESTING BOXES',
 'GLASS STAR FROSTED T-LIGHT HOLDER']

In [5]:
#Test of the previous function
diz = {}
lProva = [[5,'a'],[5,'b'],[7,'z']]
print("Transactions:")
[print(el) for el in lProva]

for row in lProva:
    if (row[0] in diz.keys()):
        diz[row[0]].append(row[1])
    else:
        diz[row[0]]=[row[1]]
print("\nAggregated transactions:")
[print(k,v) for k,v in diz.items()]
provaInvoices = list(diz.values())

Transactions:
[5, 'a']
[5, 'b']
[7, 'z']

Aggregated transactions:
5 ['a', 'b']
7 ['z']


**3 - Converting the list of invoices into a matrix**

You should now have a list (one for each invoice) of lists (each list containing the items bought for that invoice). Now, we need to convert this into a matrix form. Of the many possible formats, we will use the one expected by the Mlxtend library, which is as follows. Given an ordered list of M possible items (in this case, all possible products that can be bought), and given N itemsets (in this case, invoices), we should build a matrix of N rows and M columns. The element at the ith row and jth column should be 1 if the $i^{th}$ itemset (invoice) contains the $j^{th}$ item (product), 0 otherwise.

In [6]:
#input: a list of invoices, each invoices is a list that contains a set of items
#output: a matrix of 0(item is absent) and 1(item is present)
def matrixation(invoices):
    linvoices=[x for l in invoices for x in l]
    sortedUniqueItemsList=sorted(list(set(linvoices)))
    matrix=[]
    for row in invoices:
        tmpL=[False]*len(sortedUniqueItemsList)
        for el in row:
            tmpL[sortedUniqueItemsList.index(el)]=True
        matrix.append(tmpL)
    return (sortedUniqueItemsList, matrix)

In [7]:
#Turing the matrix in a pandas DataFrame
all_items = matrixation(invoices)[0]
pa_matrix = matrixation(invoices)[1]
df = pd.DataFrame(data=pa_matrix, columns=all_items)

In [8]:
all_items[:10]

['10 COLOUR SPACEBOY PEN',
 '12 COLOURED PARTY BALLOONS',
 '12 DAISY PEGS IN WOOD BOX',
 '12 EGG HOUSE PAINTED WOOD',
 '12 HANGING EGGS HAND PAINTED',
 '12 IVORY ROSE PEG PLACE SETTINGS',
 '12 MESSAGE CARDS WITH ENVELOPES',
 '12 PENCIL SMALL TUBE WOODLAND',
 '12 PENCILS SMALL TUBE RED RETROSPOT',
 '12 PENCILS SMALL TUBE SKULL']

In [9]:
#Test of matrixation function
prova = [['a','b','c'],['b','c'],['a','c','d'],['a','b']]
print (prova)
matrixation(prova)

[['a', 'b', 'c'], ['b', 'c'], ['a', 'c', 'd'], ['a', 'b']]


(['a', 'b', 'c', 'd'],
 [[True, True, True, False],
  [False, True, True, False],
  [True, False, True, True],
  [True, True, False, False]])

**4 - Application of the *fp_growth* function**

With the df that you defined in the previous exercise, you can now use the fp_growth function. The first argument required is the previously built DataFrame, df. The second is the minimum support (minsup), i.e. the minimum fraction of the entire dataset in which the itemset should show up for it to be considered “frequent”. Try using different values of minsup, such as 0.5, 0.1, 0.05, 0.02, 0.01. How many results do you
obtain as minsup varies? 

In [10]:
from mlxtend.frequent_patterns import fpgrowth
fis=[]
for minsup in [0.5, 0.1, 0.05, 0.02, 0.01]:    
    fi = fpgrowth(df, minsup, use_colnames=True)
    print(len(fi))
    #print(fi.to_string())
    fis.append(fi)
#To get the corrisponding itemsets use "use_colnames=True"

0
3
34
382
1923


As the *minsup* lovers the number of frequent itemset increase

**5 - minsup = 0.02**

Consider the itemsets extracted for minsup = 0.02. How many items are contained? Which ones would you be considered the most useful?

In [11]:
print("Number of itemset with minsup=0.02:",len(fis[3]))

Number of itemset with minsup=0.02: 382


In [12]:
fis[3].sort_values('support', ascending = False)

,support,itemsets
0,0.113820,(WHITE HANGING HEART T-LIGHT HOLDER)
68,0.105359,(JUMBO BAG RED RETROSPOT)
166,0.100171,(REGENCY CAKESTAND 3 TIER)
238,0.084911,(PARTY BUNTING)
30,0.078767,(LUNCH BAG RED RETROSPOT)
...,...,...
94,0.020246,(CHICK GREY HOT WATER BOTTLE)
291,0.020195,(VINTAGE CHRISTMAS BUNTING)
213,0.020095,(FOUR HOOK WHITE LOVEBIRDS)
57,0.020044,(VINTAGE SNAKES & LADDERS)


**6 - Association Rule Mining (*minsup* = 0.02)**

In [13]:
from mlxtend.frequent_patterns import association_rules
res = association_rules(fi, metric = "confidence", min_threshold=0.7)
res1 = res[["antecedents", "consequents", "support", "confidence", "lift"]]
res1
#if you want to filter on the confidence
#res2 = res1[res1["confidence"]>=0.7]

,antecedents,consequents,support,confidence,lift
0,(POPPY'S PLAYHOUSE KITCHEN),(POPPY'S PLAYHOUSE BEDROOM),0.015763,0.717890,33.698397
1,(POPPY'S PLAYHOUSE BEDROOM),(POPPY'S PLAYHOUSE KITCHEN),0.015763,0.739953,33.698397
2,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(ALARM CLOCK BAKELIKE RED),0.015763,0.748804,14.146764
3,"(JUMBO BAG PINK POLKADOT, LUNCH BAG RED RETROS...",(JUMBO BAG RED RETROSPOT),0.012540,0.770898,7.316896
4,"(JUMBO BAG PINK POLKADOT, JUMBO STORAGE BAG SUKI)",(JUMBO BAG RED RETROSPOT),0.020800,0.801942,7.611547
...,...,...,...,...,...
288,(REGENCY TEA PLATE PINK),"(REGENCY TEA PLATE GREEN, REGENCY TEA PLATE RO...",0.012842,0.836066,52.040495
289,(GARDENERS KNEELING PAD CUP OF TEA),(GARDENERS KNEELING PAD KEEP CALM),0.027498,0.720317,15.665506
290,(SET OF 3 WOODEN TREE DECORATIONS),(SET OF 3 WOODEN STOCKING DECORATION),0.011029,0.747440,46.524057
291,(SET OF 3 WOODEN TREE DECORATIONS),(SET OF 3 WOODEN HEART DECORATIONS),0.010627,0.720137,32.132653


**7 - Association rules (*minsup* = 0.01)**

In [14]:
fi = fis[4]
print(len(fi))
#print(fi.to_string())
res = association_rules(fi, metric = "confidence", min_threshold=0.85)
res1 = res[["antecedents", "consequents", "support", "confidence", "lift"]]
res1

1923


,antecedents,consequents,support,confidence,lift
0,"(JUMBO BAG BAROQUE BLACK WHITE, JUMBO BAG PIN...",(JUMBO BAG RED RETROSPOT),0.010375,0.872881,8.284862
1,"(STRAWBERRY CHARLOTTE BAG, CHARLOTTE BAG PINK ...",(RED RETROSPOT CHARLOTTE BAG),0.016418,0.860158,16.517702
2,"(STRAWBERRY CHARLOTTE BAG, CHARLOTTE BAG PINK ...",(RED RETROSPOT CHARLOTTE BAG),0.012339,0.904059,17.360731
3,"(WOODLAND CHARLOTTE BAG, STRAWBERRY CHARLOTTE ...",(RED RETROSPOT CHARLOTTE BAG),0.012188,0.920152,17.669768
4,"(WOODLAND CHARLOTTE BAG, STRAWBERRY CHARLOTTE ...",(RED RETROSPOT CHARLOTTE BAG),0.010274,0.918919,17.646087
5,"(WOODLAND CHARLOTTE BAG, STRAWBERRY CHARLOTTE ...",(RED RETROSPOT CHARLOTTE BAG),0.013094,0.858086,16.477903
6,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ...",(JUMBO BAG RED RETROSPOT),0.013497,0.867314,8.232020
7,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.013346,0.895270,33.731473
8,(HERB MARKER MINT),(HERB MARKER BASIL),0.010475,0.866667,71.109642
9,(HERB MARKER BASIL),(HERB MARKER MINT),0.010475,0.859504,71.109642
